In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Vector space model

<br>

In this report we'll code and compare different vector space model implementations (binary, tf, tf-idf and bm25). Employed data and auxiliary scripts can be found at the report's [original repository](https://github.com/Benardi/bochica).

<br>

In [2]:
from ast import literal_eval
from math import log

from IPython.display import Markdown, display, HTML
import pandas as pd

## Load Data

In [3]:
inv_index = pd.read_csv("../output/inverted_index.csv").set_index(['word'])\
            .drop('vivoÉ').drop('ônibusFoi') #remove nltk tokenization mistake
inv_index["doc_id:freq"] = inv_index["doc_id:freq"].apply(lambda x: literal_eval(x))

display(Markdown("## Inverted Index"))
display(HTML(inv_index.head(5).to_html()))

## Inverted Index

,doc_id:freq
word,
juíza,"[(0, 2), (1, 1)]"
federal,"[(0, 1), (1, 1), (2, 1), (6, 2), (14, 1), (36,..."
Ivani,"[(0, 1), (1, 1)]"
Silva,"[(0, 3), (1, 1), (5, 1), (13, 2), (25, 1), (72..."
Luz,"[(0, 3), (1, 1), (124, 1)]"


## Adding Inverse Document Frequency  (IDF)

In [4]:
# number of documents in the collection
n_docs = inv_index["doc_id:freq"].apply(lambda x: list(i[0] for i in x)).sum()
n_docs = len(set(n_docs))
display(Markdown("* Let's employ the number  of documents in this particular \
                  collection (N={}) in our calculations".format(n_docs)))

* Let's employ the number  of documents in this particular                   collection (N=249) in our calculations

In [5]:
inv_index["IDF"] = inv_index["doc_id:freq"].apply(lambda x: log((n_docs + 1)/len(x)))
inv_index.sample(5)

,doc_id:freq,IDF
word,,
babaca,"[(215, 1)]",5.521461
discutirmos,"[(28, 1)]",5.521461
disputas,"[(22, 1), (23, 1), (118, 1), (151, 1)]",4.135167
Mostra,"[(172, 1), (240, 1)]",4.828314
enganar,"[(128, 2), (231, 1)]",4.828314


<br>

***

<br>

## Strategies Implementation

### Binary Vector Space Model

In [6]:
def binary_vsm(**kwargs):
    """Applies the binary vsm to a query and document

    Applies the 'binary vector space model' to a query 
    and document, calculating a score.

    :param pandas.core.frame.DataFrame index: inverted index.
    :param list(str) query: list of terms that form a query.
    :param int doc_id: id of the document.
    
    :return: relevance score, how relevant the document is 
             for that query

    :rtype: number
    """
    
    index = kwargs["index"]
    query = kwargs["query"]
    doc_id = kwargs["doc_id"]

    result = inv_index.loc[set(query)]["doc_id:freq"]
    result = result.apply(lambda x: 0 if not list(w for w in x if w[0] == doc_id) else 1)
    result = result.sum()
    
    return result

### TF Vector Space Model

In [7]:
def tf_vsm(**kwargs):
    """Applies the tf vsm to a query and document

    Applies the 'term frequency vector space model' to a query 
    and document, calculating a score.

    :param pandas.core.frame.DataFrame index: inverted index.
    :param list(str) query: list of terms that form a query.
    :param int doc_id: id of the document.
    
    :return: relevance score, how relevant the document is 
             for that query

    :rtype: number
    """
    
    index = kwargs["index"]
    query = kwargs["query"]
    doc_id = kwargs["doc_id"]
    
    # Calculate frequency in query
    query_fd = {}
    for term in query:
        if term not in query_fd:
            query_fd[term] = 0

        query_fd[term] += 1

    
    # Get document frequency from inverted index
    term_f_in_doc = index.loc[set(query)]["doc_id:freq"]\
                      .apply(lambda x: list(w for w in x if w[0] == doc_id))\
                      .apply(lambda x: x[0][1] if not (len(x) == 0) else 0) # Extract freq 

    # Combine doc freq and query frequency to make score
    score = 0
    for term, freq in query_fd.items():
        query_score = freq 
        doc_score = term_f_in_doc.xs(term)
        score += query_score * doc_score

    return score

### TF-IDF

In [8]:
def tf_idf_vsm(**kwargs):
    """Applies the tf-idf vsm to a query and document

    Applies the 'term frequency vector inverse document frequency
    space model' to a query and document, calculating a score.

    :param pandas.core.frame.DataFrame index: inverted index.
    :param list(str) query: list of terms that form a query.
    :param int doc_id: id of the document.
    
    :return: relevance score, how relevant the document is 
             for that query

    :rtype: number
    """    
    
    index = kwargs["index"]
    query = kwargs["query"]
    doc_id = kwargs["doc_id"]
    
    # Calculate frequency in query
    query_fd = {}
    for term in query:
        if term not in query_fd:
            query_fd[term] = 0

        query_fd[term] += 1

    # Get document frequency from inverted index
    term_f_in_doc = index.loc[set(query)]
    term_f_in_doc["doc_id:freq"] = term_f_in_doc["doc_id:freq"].\
                                   apply(lambda x: list(w for w in x if w[0] == doc_id))
    term_f_in_doc["doc_id:freq"] = term_f_in_doc["doc_id:freq"].\
                                   apply(lambda x: x[0][1] if not (len(x) == 0) else 0)

    # Combine doc freq and query frequency to make score
    score = 0
    for term, freq in query_fd.items():
        query_score = freq 
        doc_score = term_f_in_doc.xs(term)['doc_id:freq']
        idf_score = term_f_in_doc.xs(term)['IDF']
        score += query_score * doc_score * idf_score
        
    return score

### BM25 Vector Space Model

In [9]:
def bm25_vsm(**kwargs):
    """Applies the bm25 vsm to a query and document

    Applies the 'Best Matching 25 space model' to a query 
    and document, calculating a score.

    :param pandas.core.frame.DataFrame index: inverted index.
    :param list(str) query: list of terms that form a query.
    :param int doc_id: id of the document.
    :param int k: term frequency saturation factor
    
    :return: relevance score, how relevant the document is 
             for that query

    :rtype: number
    """    
    
    index = kwargs["index"]
    query = kwargs["query"]
    doc_id = kwargs["doc_id"]
    k = kwargs["k"]
    
    # Calculate frequency in query
    query_fd = {}
    for term in query:
        if term not in query_fd:
            query_fd[term] = 0

        query_fd[term] += 1

    # Get document frequency from inverted index
    term_f_in_doc = index.reset_index().set_index('word').loc[set(query)]
    term_f_in_doc["doc_id:freq"] = term_f_in_doc["doc_id:freq"].\
                                   apply(lambda x: list(w for w in x if w[0] == doc_id))
    term_f_in_doc["doc_id:freq"] = term_f_in_doc["doc_id:freq"].\
                                   apply(lambda x: x[0][1] if not (len(x) == 0) else 0)

    # Combine doc freq and query frequency to make score
    score = 0
    for term, freq in query_fd.items():
        query_score = freq 
        doc_count = term_f_in_doc.xs(term)['doc_id:freq']
        doc_score = 0 if doc_count == 0 else (doc_count * (k + 1)) / (doc_count + k)
        idf_score = term_f_in_doc.xs(term)['IDF']
        score += query_score * doc_score * idf_score

    return score

### Strategy Wrapper

In [10]:
from bisect import insort_left


def get_n_best_docs(**kwargs):
    """Retrieves n most relevant docs according to a strategy

    Retrieves the n most relevant documents in a given collection
    according to a given strategy.

    :param pandas.core.frame.DataFrame index: inverted index.
    :param list(str) query: list of terms that form a query.
    :param list(int) docs: All documents to be evaluated 
                           (extracted from index if not specified).
    :param function strategy: vsm strategy (its params must be 
                              supplied as well)
    
    :return: list of document ids by rank in decrescent order

    :rtype: list((number,number))
    """   

    n = kwargs.pop("n")
    strategy = kwargs.pop("strategy")
    
    if "docs" in kwargs:
        docs = kwargs.pop("docs")
    else:        
        docs = index["doc_id:freq"].apply(lambda x: list(w[0] for w in x)).sum()
        docs = list(set(docs))
        
    rank = []
    for doc in docs:
        score = strategy(doc_id=doc,**kwargs)
        insort_left(rank, (score,doc))
        rank = rank[-n:]

    return rank

<br>

***

<br>

## Strategy Comparison

We'll use the inverted index's source data to retrieve the text, title and url of the documents. 

These will be inspected to understand the results we arrive to.

In [11]:
source = pd.read_csv("../output/results.csv")
display(Markdown("##### Inverted Index's source data"))
source["title"].head()

##### Inverted Index's source data

0    “A sociedade foi Rubens Paiva  não os facínora...
1    Justiça suspende decisão que proibia Forças Ar...
2    Governo Bolsonaro prega “negacionismo históric...
3    Quando os pais de Gabo perceberam que tinham u...
4    Rádios canadenses banem músicas de Michael Jac...
Name: title, dtype: object

Let's gather the id of all documents to supply the algorithms:

In [12]:
all_docs = inv_index["doc_id:freq"].apply(lambda x: list(w[0] for w in x)).sum()
all_docs = list(set(all_docs))
pd.DataFrame(all_docs, columns=["docs"]).describe()

,docs
count,249.000000
mean,124.000000
std,72.024301
min,0.000000
25%,62.000000
50%,124.000000
75%,186.000000
max,248.000000


In [13]:
query1 = ["1964","golpe","ditadura","Bolsonaro","militar"]
query2 = ["árabe","islâmico"]
query3 = ["feminismo","mulher","feminista"]

display(Markdown("##### We'll employ the three following queries:"))
display(Markdown("* Query 1 : {}".format(', '.join(query1))))
display(Markdown("* Query 2 : {}".format(', '.join(query2))))
display(Markdown("* Query 3 : {}".format(', '.join(query3))))

##### We'll employ the three following queries:

* Query 1 : 1964, golpe, ditadura, Bolsonaro, militar

* Query 2 : árabe, islâmico

* Query 3 : feminismo, mulher, feminista

<br>

### K-effect on BM25

The strategy BM25 has a parameter *k* which is said to impact in its results.

Let's observe the effect of different values of *k* in the algorithm's results.

In [14]:
strategies = [bm25_vsm,bm25_vsm,bm25_vsm,bm25_vsm]
columns = ["BM25(k=0)","BM25(k=1)","BM25(k=10)","BM25(k=100)"]
k_values =[0,1,10,100]
rows = []

for idx in range(0,len(strategies)):
    top_10 = get_n_best_docs(n=5, k=k_values[idx], docs=all_docs,index=inv_index,
                             query=query1, strategy=strategies[idx])
    
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
pd.set_option('max_colwidth', -1)
display(Markdown("##### Top 5 documents by k for 'Query 1' (title)"))
display(HTML(pd.DataFrame(list(zip(*[list(map(lambda x: source.xs(x)['title'], el)) for el in rows])),
                          columns=columns).to_html(index=False)))

##### Top 5 documents by k for 'Query 1' (title)

BM25(k=0),BM25(k=1),BM25(k=10),BM25(k=100)
Atos pró e contra ditadura militar terminam em confrontos em São Paulo,Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”,Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”,Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”
“Não me arrependo de nada”,Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe,Bolsonaro manda festejar o crime,Bolsonaro manda festejar o crime
Um Brasil de dedo em riste às portas do Mackenzie,Bolsonaro manda festejar o crime,Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe,Quem mandou matar Marielle? E por quê?
Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe,“A sociedade foi Rubens Paiva não os facínoras que o mataram”,Um Brasil de dedo em riste às portas do Mackenzie,Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe
A fuga simbólica de Bolsonaro para Israel no 55º aniversário da ditadura brasileira,Um Brasil de dedo em riste às portas do Mackenzie,Governo Bolsonaro prega “negacionismo histórico” sobre a ditadura,Um Brasil de dedo em riste às portas do Mackenzie


* An improvement in the results is visible as k increases
    + The documents retrieved for k=1 seem more relevant than those retrieve with k=0
    + The documents retrieved with k=1 and k=10 seem to be approximately as relevant
    + In documents retrieved with k=100 we start to see a decline in relevance (The document about 'Marielle' isn't as directly related to the query as the rest)

In [15]:
pd.set_option('max_colwidth', -1)
display(Markdown("##### Top 5 documents by k for 'Query 1' (url)"))
display(HTML(pd.DataFrame(list(zip(*[list(map(lambda x: source.xs(x)['url'], el)) for el in rows])),
                          columns=columns).to_html(index=False)))

##### Top 5 documents by k for 'Query 1' (url)

BM25(k=0),BM25(k=1),BM25(k=10),BM25(k=100)
https://brasil.elpais.com/brasil/2019/04/01/politica/1554118634_296372.html,https://brasil.elpais.com/brasil/2019/03/30/politica/1553984610_147330.html,https://brasil.elpais.com/brasil/2019/03/30/politica/1553984610_147330.html,https://brasil.elpais.com/brasil/2019/03/30/politica/1553984610_147330.html
https://brasil.elpais.com/brasil/2019/03/28/politica/1553789942_315053.html,https://brasil.elpais.com/brasil/2019/03/26/politica/1553609505_570456.html,https://brasil.elpais.com/brasil/2019/03/27/opinion/1553688411_058227.html,https://brasil.elpais.com/brasil/2019/03/27/opinion/1553688411_058227.html
https://brasil.elpais.com/brasil/2019/03/27/politica/1553717682_889482.html,https://brasil.elpais.com/brasil/2019/03/27/opinion/1553688411_058227.html,https://brasil.elpais.com/brasil/2019/03/26/politica/1553609505_570456.html,https://brasil.elpais.com/brasil/2019/03/13/opinion/1552485039_897963.html
https://brasil.elpais.com/brasil/2019/03/26/politica/1553609505_570456.html,https://brasil.elpais.com/brasil/2019/03/26/politica/1553638410_317117.html,https://brasil.elpais.com/brasil/2019/03/27/politica/1553717682_889482.html,https://brasil.elpais.com/brasil/2019/03/26/politica/1553609505_570456.html
https://brasil.elpais.com/brasil/2019/04/01/opinion/1554078976_839038.html,https://brasil.elpais.com/brasil/2019/03/27/politica/1553717682_889482.html,https://brasil.elpais.com/brasil/2019/04/05/politica/1554419295_939718.html,https://brasil.elpais.com/brasil/2019/03/27/politica/1553717682_889482.html


* Although the topic of military dictatorship and Bolsonaro is mentioned/discussed in the article about 'Marielle' (k=100) it isn't as central to the article as in other topics.
    + The results for (k=100) are of good quality but we start to see a sightly fall in relevance.

In [16]:
strategies = [bm25_vsm,bm25_vsm,bm25_vsm,bm25_vsm]
columns = ["BM25(k=0)","BM25(k=1)","BM25(k=10)","BM25(k=100)"]
k_values =[0,1,10,100]
rows = []

for idx in range(0,len(strategies)):
    top_10 = get_n_best_docs(n=5, k=k_values[idx], docs=all_docs,index=inv_index,
                             query=query2, strategy=strategies[idx])
    
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
pd.set_option('max_colwidth', -1)
display(Markdown("##### Top 5 documents by k for 'Query 2' (title)"))
display(HTML(pd.DataFrame(list(zip(*[list(map(lambda x: source.xs(x)['title'], el)) for el in rows])),
                          columns=columns).to_html(index=False)))

##### Top 5 documents by k for 'Query 2' (title)

BM25(k=0),BM25(k=1),BM25(k=10),BM25(k=100)
Bolsonaro troca embaixada por escritório em Jerusalém mas não evita retaliação palestina,Bolsonaro troca embaixada por escritório em Jerusalém mas não evita retaliação palestina,As três espanholas do Estado Islâmico: “Só queremos ir embora”,As três espanholas do Estado Islâmico: “Só queremos ir embora”
As três espanholas do Estado Islâmico: “Só queremos ir embora”,As três espanholas do Estado Islâmico: “Só queremos ir embora”,Bolsonaro troca embaixada por escritório em Jerusalém mas não evita retaliação palestina,Bolsonaro troca embaixada por escritório em Jerusalém mas não evita retaliação palestina
Brunei começa a punir com apedrejamento o adultério e a homossexualidade,Brunei começa a punir com apedrejamento o adultério e a homossexualidade,Brunei começa a punir com apedrejamento o adultério e a homossexualidade,Brunei começa a punir com apedrejamento o adultério e a homossexualidade
Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”,Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”,Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”,Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”
Dois aviões russos chegam à Venezuela com uma centena de militares,Dois aviões russos chegam à Venezuela com uma centena de militares,Dois aviões russos chegam à Venezuela com uma centena de militares,Dois aviões russos chegam à Venezuela com uma centena de militares


* The biggger the k the more relevant the retrieved documents
* Results for k=10 and k=100 are the same

In [17]:
pd.set_option('max_colwidth', -1)
display(Markdown("##### Top 5 documents by k for 'Query 2' (url)"))
display(HTML(pd.DataFrame(list(zip(*[list(map(lambda x: source.xs(x)['url'], el)) for el in rows])),
                          columns=columns).to_html(index=False)))

##### Top 5 documents by k for 'Query 2' (url)

BM25(k=0),BM25(k=1),BM25(k=10),BM25(k=100)
https://brasil.elpais.com/brasil/2019/03/31/politica/1554060705_325198.html,https://brasil.elpais.com/brasil/2019/03/31/politica/1554060705_325198.html,https://brasil.elpais.com/brasil/2019/04/03/internacional/1554297555_947058.html,https://brasil.elpais.com/brasil/2019/04/03/internacional/1554297555_947058.html
https://brasil.elpais.com/brasil/2019/04/03/internacional/1554297555_947058.html,https://brasil.elpais.com/brasil/2019/04/03/internacional/1554297555_947058.html,https://brasil.elpais.com/brasil/2019/03/31/politica/1554060705_325198.html,https://brasil.elpais.com/brasil/2019/03/31/politica/1554060705_325198.html
https://brasil.elpais.com/brasil/2019/04/03/internacional/1554284355_385703.html,https://brasil.elpais.com/brasil/2019/04/03/internacional/1554284355_385703.html,https://brasil.elpais.com/brasil/2019/04/03/internacional/1554284355_385703.html,https://brasil.elpais.com/brasil/2019/04/03/internacional/1554284355_385703.html
https://brasil.elpais.com/brasil/2019/03/20/politica/1553037448_213932.html,https://brasil.elpais.com/brasil/2019/03/20/politica/1553037448_213932.html,https://brasil.elpais.com/brasil/2019/03/20/politica/1553037448_213932.html,https://brasil.elpais.com/brasil/2019/03/20/politica/1553037448_213932.html
https://brasil.elpais.com/brasil/2019/03/24/internacional/1553458236_783517.html,https://brasil.elpais.com/brasil/2019/03/24/internacional/1553458236_783517.html,https://brasil.elpais.com/brasil/2019/03/24/internacional/1553458236_783517.html,https://brasil.elpais.com/brasil/2019/03/24/internacional/1553458236_783517.html


* The articles themselves are consistent with the results gathered through their titles.

In [18]:
strategies = [bm25_vsm,bm25_vsm,bm25_vsm,bm25_vsm]
columns = ["BM25(k=0)","BM25(k=1)","BM25(k=10)","BM25(k=100)"]
k_values =[0,1,10,100]
rows = []

for idx in range(0,len(strategies)):
    top_10 = get_n_best_docs(n=5, k=k_values[idx], docs=all_docs,index=inv_index,
                             query=query3, strategy=strategies[idx])
    
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
pd.set_option('max_colwidth', -1)
display(Markdown("##### Top 5 documents by k for 'Query 3' (title)"))
display(HTML(pd.DataFrame(list(zip(*[list(map(lambda x: source.xs(x)['title'], el)) for el in rows])),
                          columns=columns).to_html(index=False)))

##### Top 5 documents by k for 'Query 3' (title)

BM25(k=0),BM25(k=1),BM25(k=10),BM25(k=100)
A ética da solidariedade feminista,A feminista Susan Sontag contra Norman Mailer,A feminista Susan Sontag contra Norman Mailer,A feminista Susan Sontag contra Norman Mailer
A feminista Susan Sontag contra Norman Mailer,A ética da solidariedade feminista,Não há ‘backlash’ provocado pelo feminismo,Não há ‘backlash’ provocado pelo feminismo
Igualdade radical,Igualdade radical,A ética da solidariedade feminista,A ética da solidariedade feminista
Atos pró e contra ditadura militar terminam em confrontos em São Paulo,Não há ‘backlash’ provocado pelo feminismo,Igualdade radical,O que a fama fez com Susan Boyle a mulher que revolucionou a música a televisão e as redes sociais
Não há ‘backlash’ provocado pelo feminismo,Atos pró e contra ditadura militar terminam em confrontos em São Paulo,O que a fama fez com Susan Boyle a mulher que revolucionou a música a televisão e as redes sociais,Igualdade radical


* k = 1 represents a clear improvement over k=0
* Starting with k=10 we see an introduction of a less relevant document (about Susan Boyle)
    + With k=100 this less relevant document takes the place of a relevant document (about Igualdade radical)

In [19]:
pd.set_option('max_colwidth', -1)
display(Markdown("##### Top 5 documents by k for 'Query 3' (url)"))
display(HTML(pd.DataFrame(list(zip(*[list(map(lambda x: source.xs(x)['url'], el)) for el in rows])),
                          columns=columns).to_html(index=False)))

##### Top 5 documents by k for 'Query 3' (url)

BM25(k=0),BM25(k=1),BM25(k=10),BM25(k=100)
https://brasil.elpais.com/brasil/2019/03/01/opinion/1551471471_656137.html,https://brasil.elpais.com/brasil/2019/03/05/cultura/1551786137_303043.html,https://brasil.elpais.com/brasil/2019/03/05/cultura/1551786137_303043.html,https://brasil.elpais.com/brasil/2019/03/05/cultura/1551786137_303043.html
https://brasil.elpais.com/brasil/2019/03/05/cultura/1551786137_303043.html,https://brasil.elpais.com/brasil/2019/03/01/opinion/1551471471_656137.html,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551994753_797742.html,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551994753_797742.html
https://brasil.elpais.com/brasil/2019/03/07/opinion/1551972628_054337.html,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551972628_054337.html,https://brasil.elpais.com/brasil/2019/03/01/opinion/1551471471_656137.html,https://brasil.elpais.com/brasil/2019/03/01/opinion/1551471471_656137.html
https://brasil.elpais.com/brasil/2019/04/01/politica/1554118634_296372.html,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551994753_797742.html,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551972628_054337.html,https://brasil.elpais.com/brasil/2019/04/04/cultura/1554380845_480636.html
https://brasil.elpais.com/brasil/2019/03/07/opinion/1551994753_797742.html,https://brasil.elpais.com/brasil/2019/04/01/politica/1554118634_296372.html,https://brasil.elpais.com/brasil/2019/04/04/cultura/1554380845_480636.html,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551972628_054337.html


* The document about Susan Boyle is indeed less relevant

> Up to moderate values (k=1 to k=10) an increase in k overall translates into more relevant results.

> If increased too much (e.g. k=100) we see either no change or the addition of less relevant results.

<br>

### Relevance of results by strategy

In view of the previous section's results we shall use k=10 for the algorithm BM25 

In [20]:
strategies = [binary_vsm,tf_vsm,tf_idf_vsm,bm25_vsm]
columns = ["Binary","TF","TF-IDF","BM25(k=10)"]
k_values =[None,None,None,10]
rows = []

for idx in range(0,len(strategies)):
    top_10 = get_n_best_docs(n=5, k=k_values[idx], docs=all_docs,index=inv_index,
                             query=query1, strategy=strategies[idx])
    
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)

rst_q1 = rows
pd.set_option('max_colwidth', -1)
display(Markdown("##### Top 5 best docs by strategy for query1 (title)"))
display(HTML(pd.DataFrame(list(zip(*[list(map(lambda x: source.xs(x)['title'], el)) for el in rows])),
                          columns=columns).to_html(index=False)))

##### Top 5 best docs by strategy for query1 (title)

Binary,TF,TF-IDF,BM25(k=10)
Atos pró e contra ditadura militar terminam em confrontos em São Paulo,Quem mandou matar Marielle? E por quê?,Bolsonaro manda festejar o crime,Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”
“Não me arrependo de nada”,Bolsonaro manda festejar o crime,Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”,Bolsonaro manda festejar o crime
Um Brasil de dedo em riste às portas do Mackenzie,Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”,Quem mandou matar Marielle? E por quê?,Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe
Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe,Dois assessores de Bolsonaro doaram mais de 100.000 reais a campanhas da família,Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe,Um Brasil de dedo em riste às portas do Mackenzie
A fuga simbólica de Bolsonaro para Israel no 55º aniversário da ditadura brasileira,Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe,Um Brasil de dedo em riste às portas do Mackenzie,Governo Bolsonaro prega “negacionismo histórico” sobre a ditadura


* The strategy that provided the best results, in terms of titles, was BM25(k=1)
    + Its documents e.g. cite Bolsonaro directly 

In [21]:
pd.set_option('max_colwidth', -1)
display(Markdown("##### Top 5 best docs by strategy for query1 (url)"))
display(HTML(pd.DataFrame(list(zip(*[list(map(lambda x: source.xs(x)['url'], el)) for el in rows])),
                          columns=columns).to_html(index=False)))

##### Top 5 best docs by strategy for query1 (url)

Binary,TF,TF-IDF,BM25(k=10)
https://brasil.elpais.com/brasil/2019/04/01/politica/1554118634_296372.html,https://brasil.elpais.com/brasil/2019/03/13/opinion/1552485039_897963.html,https://brasil.elpais.com/brasil/2019/03/27/opinion/1553688411_058227.html,https://brasil.elpais.com/brasil/2019/03/30/politica/1553984610_147330.html
https://brasil.elpais.com/brasil/2019/03/28/politica/1553789942_315053.html,https://brasil.elpais.com/brasil/2019/03/27/opinion/1553688411_058227.html,https://brasil.elpais.com/brasil/2019/03/30/politica/1553984610_147330.html,https://brasil.elpais.com/brasil/2019/03/27/opinion/1553688411_058227.html
https://brasil.elpais.com/brasil/2019/03/27/politica/1553717682_889482.html,https://brasil.elpais.com/brasil/2019/03/30/politica/1553984610_147330.html,https://brasil.elpais.com/brasil/2019/03/13/opinion/1552485039_897963.html,https://brasil.elpais.com/brasil/2019/03/26/politica/1553609505_570456.html
https://brasil.elpais.com/brasil/2019/03/26/politica/1553609505_570456.html,https://brasil.elpais.com/brasil/2019/03/21/politica/1553173153_987739.html,https://brasil.elpais.com/brasil/2019/03/26/politica/1553609505_570456.html,https://brasil.elpais.com/brasil/2019/03/27/politica/1553717682_889482.html
https://brasil.elpais.com/brasil/2019/04/01/opinion/1554078976_839038.html,https://brasil.elpais.com/brasil/2019/03/26/politica/1553609505_570456.html,https://brasil.elpais.com/brasil/2019/03/27/politica/1553717682_889482.html,https://brasil.elpais.com/brasil/2019/04/05/politica/1554419295_939718.html


Inspecting the articles we can see that BM25(k=1) really is the strategy with the best results
* TF retrieved documents who talk about Bolsonaro but not directly about dictatorship
* TF-IDF retrieved the document about 'Marielle' where as mentioned before Bolsonaro/Dictatorship isn't the central topic

In [22]:
strategies = [binary_vsm,tf_vsm,tf_idf_vsm,bm25_vsm]
columns = ["Binary","TF","TF-IDF","BM25(k=10)"]
k_values =[None,None,None,10]
rows = []

for idx in range(0,len(strategies)):
    top_10 = get_n_best_docs(n=5, k=k_values[idx], docs=all_docs,index=inv_index,
                             query=query2, strategy=strategies[idx])
    
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)

rst_q2 = rows
pd.set_option('max_colwidth', -1)
display(Markdown("##### Top 5 best docs by strategy for query2 (title)"))
display(HTML(pd.DataFrame(list(zip(*[list(map(lambda x: source.xs(x)['title'], el)) for el in rows])),
                          columns=columns).to_html(index=False)))

##### Top 5 best docs by strategy for query2 (title)

Binary,TF,TF-IDF,BM25(k=10)
Bolsonaro troca embaixada por escritório em Jerusalém mas não evita retaliação palestina,As três espanholas do Estado Islâmico: “Só queremos ir embora”,As três espanholas do Estado Islâmico: “Só queremos ir embora”,As três espanholas do Estado Islâmico: “Só queremos ir embora”
As três espanholas do Estado Islâmico: “Só queremos ir embora”,Bolsonaro troca embaixada por escritório em Jerusalém mas não evita retaliação palestina,Bolsonaro troca embaixada por escritório em Jerusalém mas não evita retaliação palestina,Bolsonaro troca embaixada por escritório em Jerusalém mas não evita retaliação palestina
Brunei começa a punir com apedrejamento o adultério e a homossexualidade,Brunei começa a punir com apedrejamento o adultério e a homossexualidade,Brunei começa a punir com apedrejamento o adultério e a homossexualidade,Brunei começa a punir com apedrejamento o adultério e a homossexualidade
Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”,Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”,Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”,Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”
Dois aviões russos chegam à Venezuela com uma centena de militares,Dois aviões russos chegam à Venezuela com uma centena de militares,Dois aviões russos chegam à Venezuela com uma centena de militares,Dois aviões russos chegam à Venezuela com uma centena de militares


* Excluding Binary all strategies performed the same
* Binary had a worse result, it's top document is less relevant than that of the other strategies

In [23]:
pd.set_option('max_colwidth', -1)
display(Markdown("##### Top 5 best docs by strategy for query2 (url)"))
display(HTML(pd.DataFrame(list(zip(*[list(map(lambda x: source.xs(x)['url'], el)) for el in rows])),
                          columns=columns).to_html(index=False)))

##### Top 5 best docs by strategy for query2 (url)

Binary,TF,TF-IDF,BM25(k=10)
https://brasil.elpais.com/brasil/2019/03/31/politica/1554060705_325198.html,https://brasil.elpais.com/brasil/2019/04/03/internacional/1554297555_947058.html,https://brasil.elpais.com/brasil/2019/04/03/internacional/1554297555_947058.html,https://brasil.elpais.com/brasil/2019/04/03/internacional/1554297555_947058.html
https://brasil.elpais.com/brasil/2019/04/03/internacional/1554297555_947058.html,https://brasil.elpais.com/brasil/2019/03/31/politica/1554060705_325198.html,https://brasil.elpais.com/brasil/2019/03/31/politica/1554060705_325198.html,https://brasil.elpais.com/brasil/2019/03/31/politica/1554060705_325198.html
https://brasil.elpais.com/brasil/2019/04/03/internacional/1554284355_385703.html,https://brasil.elpais.com/brasil/2019/04/03/internacional/1554284355_385703.html,https://brasil.elpais.com/brasil/2019/04/03/internacional/1554284355_385703.html,https://brasil.elpais.com/brasil/2019/04/03/internacional/1554284355_385703.html
https://brasil.elpais.com/brasil/2019/03/20/politica/1553037448_213932.html,https://brasil.elpais.com/brasil/2019/03/20/politica/1553037448_213932.html,https://brasil.elpais.com/brasil/2019/03/20/politica/1553037448_213932.html,https://brasil.elpais.com/brasil/2019/03/20/politica/1553037448_213932.html
https://brasil.elpais.com/brasil/2019/03/24/internacional/1553458236_783517.html,https://brasil.elpais.com/brasil/2019/03/24/internacional/1553458236_783517.html,https://brasil.elpais.com/brasil/2019/03/24/internacional/1553458236_783517.html,https://brasil.elpais.com/brasil/2019/03/24/internacional/1553458236_783517.html


* The articles themselves are consistent with the results gathered through their titles.

In [24]:
strategies = [binary_vsm,tf_vsm,tf_idf_vsm,bm25_vsm]
columns = ["Binary","TF","TF-IDF","BM25(k=10)"]
k_values =[None,None,None,10]
rows = []

for idx in range(0,len(strategies)):
    top_10 = get_n_best_docs(n=5, k=k_values[idx], docs=all_docs,index=inv_index,
                             query=query3, strategy=strategies[idx])
    
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)

rst_q3 = rows
pd.set_option('max_colwidth', -1)
display(Markdown("##### Top 5 best docs by strategy for query3 (title)"))
display(HTML(pd.DataFrame(list(zip(*[list(map(lambda x: source.xs(x)['title'], el)) for el in rows])),
                          columns=columns).to_html(index=False)))

##### Top 5 best docs by strategy for query3 (title)

Binary,TF,TF-IDF,BM25(k=10)
A ética da solidariedade feminista,A feminista Susan Sontag contra Norman Mailer,A feminista Susan Sontag contra Norman Mailer,A feminista Susan Sontag contra Norman Mailer
A feminista Susan Sontag contra Norman Mailer,O que a fama fez com Susan Boyle a mulher que revolucionou a música a televisão e as redes sociais,Não há ‘backlash’ provocado pelo feminismo,Não há ‘backlash’ provocado pelo feminismo
Atos pró e contra ditadura militar terminam em confrontos em São Paulo,Não há ‘backlash’ provocado pelo feminismo,O que a fama fez com Susan Boyle a mulher que revolucionou a música a televisão e as redes sociais,A ética da solidariedade feminista
Não há ‘backlash’ provocado pelo feminismo,As três espanholas do Estado Islâmico: “Só queremos ir embora”,A ética da solidariedade feminista,Igualdade radical
Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”,Quem mandou matar Marielle? E por quê?,Igualdade radical,O que a fama fez com Susan Boyle a mulher que revolucionou a música a televisão e as redes sociais


* Binary retrieved documents of little relevance (e.g. about Brazilian dictatorship)
* The document of little relevance about Susan Boyle was featured by TF, TF-IDF and BM25(k=10)
    + It was given a higher ranking by TF and TF-IDF than by BM25(k=10)
    
* Overall BM25(k=10) performed better

In [25]:
pd.set_option('max_colwidth', -1)
display(Markdown("##### Top 5 best docs by strategy for query3 (url)"))
display(HTML(pd.DataFrame(list(zip(*[list(map(lambda x: source.xs(x)['url'], el)) for el in rows])),
                          columns=columns).to_html(index=False)))

##### Top 5 best docs by strategy for query3 (url)

Binary,TF,TF-IDF,BM25(k=10)
https://brasil.elpais.com/brasil/2019/03/01/opinion/1551471471_656137.html,https://brasil.elpais.com/brasil/2019/03/05/cultura/1551786137_303043.html,https://brasil.elpais.com/brasil/2019/03/05/cultura/1551786137_303043.html,https://brasil.elpais.com/brasil/2019/03/05/cultura/1551786137_303043.html
https://brasil.elpais.com/brasil/2019/03/05/cultura/1551786137_303043.html,https://brasil.elpais.com/brasil/2019/04/04/cultura/1554380845_480636.html,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551994753_797742.html,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551994753_797742.html
https://brasil.elpais.com/brasil/2019/04/01/politica/1554118634_296372.html,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551994753_797742.html,https://brasil.elpais.com/brasil/2019/04/04/cultura/1554380845_480636.html,https://brasil.elpais.com/brasil/2019/03/01/opinion/1551471471_656137.html
https://brasil.elpais.com/brasil/2019/03/07/opinion/1551994753_797742.html,https://brasil.elpais.com/brasil/2019/04/03/internacional/1554297555_947058.html,https://brasil.elpais.com/brasil/2019/03/01/opinion/1551471471_656137.html,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551972628_054337.html
https://brasil.elpais.com/brasil/2019/03/30/politica/1553984610_147330.html,https://brasil.elpais.com/brasil/2019/03/13/opinion/1552485039_897963.html,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551972628_054337.html,https://brasil.elpais.com/brasil/2019/04/04/cultura/1554380845_480636.html


* The articles themselves are consistent with the results gathered through their titles.

> All queries' results put together, **BM25(k=10)** *performed better than the other strategies* (retrieved the most relevant documents) 

In [26]:
def intersection(lst1, lst2): 
    return [value for value in lst1 if value in lst2] 
     
def jaccard_index(A,B):
    n_A = len(A)
    n_B = len(B)
    n_intxn = len(intersection(A,B))
    return n_intxn/ ((n_A + n_B) - n_intxn)

<br>

***

<br>

## Similarity between strategies  (Jaccard Index)

In [27]:
columns = ["target","Binary","TF","TF-IDF","BM25(k=10)"]
target = ["Binary","TF","TF-IDF","BM25(k=10)"]

rows = []
for idx, el in enumerate(rst_q1):
    row = [jaccard_index(el,rst_q1[idx]) for el in rst_q1]
    row = [target[idx]] + row
    rows.append(tuple(row))

display(Markdown("##### Jaccard Index among strategies (Query 1)"))
display(HTML(pd.DataFrame(rows,columns=columns).to_html(index=False)))

##### Jaccard Index among strategies (Query 1)

target,Binary,TF,TF-IDF,BM25(k=10)
Binary,1.000000,0.111111,0.250000,0.250000
TF,0.111111,1.000000,0.666667,0.428571
TF-IDF,0.250000,0.666667,1.000000,0.666667
BM25(k=10),0.250000,0.428571,0.666667,1.000000


* Binary results are not very similar to the results of the other strategies
* TF-IDF results are moderately similar to those of BM25 and TF
    + This make sense as this strategy is much like an intermediary stage between the other two

In [28]:
columns = ["target","Binary","TF","TF-IDF","BM25(k=10)"]
target = ["Binary","TF","TF-IDF","BM25(k=10)"]

rows = []
for idx, el in enumerate(rst_q2):
    row = [jaccard_index(el,rst_q2[idx]) for el in rst_q2]
    row = [target[idx]] + row
    rows.append(tuple(row))

display(Markdown("##### Jaccard Index among strategies (Query 2)"))
display(HTML(pd.DataFrame(rows,columns=columns).to_html(index=False)))

##### Jaccard Index among strategies (Query 2)

target,Binary,TF,TF-IDF,BM25(k=10)
Binary,1.0,1.0,1.0,1.0
TF,1.0,1.0,1.0,1.0
TF-IDF,1.0,1.0,1.0,1.0
BM25(k=10),1.0,1.0,1.0,1.0


* As seen before, the results differ in terms of which document occupies which position in the rank
    + Jaccard Index doesn't represent this information and for this reason all strategies are identical according to it

In [29]:
columns = ["target","Binary","TF","TF-IDF","BM25(k=10)"]
target = ["Binary","TF","TF-IDF","BM25(k=10)"]

rows = []
for idx, el in enumerate(rst_q3):
    row = [jaccard_index(el,rst_q3[idx]) for el in rst_q3]
    row = [target[idx]] + row
    rows.append(tuple(row))

display(Markdown("##### Jaccard Index among strategies (Query 3)"))
display(HTML(pd.DataFrame(rows,columns=columns).to_html(index=False)))

##### Jaccard Index among strategies (Query 3)

target,Binary,TF,TF-IDF,BM25(k=10)
Binary,1.000000,0.250000,0.428571,0.428571
TF,0.250000,1.000000,0.428571,0.428571
TF-IDF,0.428571,0.428571,1.000000,1.000000
BM25(k=10),0.428571,0.428571,1.000000,1.000000


* TF-IDF and BM25(k=10)	are identical in terms of Jaccard Index
    + Once again, Jaccard disregard ranking order
+ Binary and TF are equaly similar to TF-IDF/BM25(k=10)
    + Binary and TF however are even less similar to each other

> Using Jaccard Index we lose the information of which documents occupy which position in the rank (quite important, come to think of it), see the case of Query 2 for instance. 

> Overall the Binary VSM's results differed quite a lot from the rest, this is a reflect of all the information this particular implementation disregards.

<br>

***

<br>
    
## Summary

<br>

> An increase in K meant improvement in BM25's results (up to a point) for this particular collection

> BM25 with k=10 had the best overall results for the 3 queries considered in this report

> Jaccard Index showed that Binary was less similar to the other implementations, which might be attributed to its naivety